# CS525 FINAL PROJECT
### GROUP MEMBERS: Ted Monyak, Jack Forman




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os


# Introduction

### Dataset

### Networks

# Results

### Training

### Simple CNN
<style>
img {
    display: block;
    margin-left: 0;
    margin-right: auto;
    width: 30%;
    height: 30%;
}
</style>


Discussion about training accuracy  
![image](Model_SimpleCNN/TrainingAccuracy.png)

Discussion about validation accuracy  
![image](Model_SimpleCNN/ValAccuracy.png)

Discussion about testing accuracy  
![image](Model_SimpleCNN/TestingAccuracy.png)

### Deeper CCN
<style>
img {
    display: block;
    margin-left: 0;
    margin-right: auto;
    width: 30%;
    height: 30%;
}
</style>


Discussion about training accuracy  
![image](Model_DeepCNN/TrainingAccuracy.png)

Discussion about validation accuracy  
![image](Model_DeepCNN/ValAccuracy.png)

Discussion about testing accuracy  
![image](Model_DeepCNN/TestingAccuracy.png)


### Transformer
<style>
img {
    display: block;
    margin-left: 0;
    margin-right: auto;
    width: 30%;
    height: 30%;
}
</style>


Discussion about training accuracy  
![image](Model_DNATranformer/TrainingAccuracy.png)

Discussion about validation accuracy  
![image](Model_DNATranformer/ValAccuracy.png)

Discussion about testing accuracy  
![image](Model_DNATranformer/TestingAccuracy.png)

# Discussion